In [82]:
import pandas as pd
from sentinelhub import WebFeatureService, BBox, CRS, DataSource, SHConfig
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import pandas as pd
from collections import OrderedDict
from sentinelhub import AwsTileRequest
from env_vars import sentinel_username,sentinel_password


In [83]:
df = pd.read_csv("./data/labelled/labels_Misha.csv")

In [93]:
df

,image,lat,long,zoom,country,concession,tile date,tile AWS path,tile SciHub link,Comment
0,1,2.64796,17.26004,@300m,Republic of Congo,Ipendja,7-Feb-20,s3://sentinel-s2-l2a/tiles/33/N/YC/2020/2/7/0,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,Not on 'logging roads'
1,2,1.06496,13.33962,@300m,Gabon,Sunry Gabon Nord-Est,4-Jan-18,s3://sentinel-s2-l2a/tiles/33/N/TB/2018/1/4/0,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,NaN
2,3,3.39416,17.39393,@500m,Republic of Congo,Mokabi-Dzanga,7-Feb-20,s3://sentinel-s2-l2a/tiles/33/N/YC/2020/2/7/0,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,NaN
3,4,3.28766,17.48586,@500m,Republic of Congo,Mokabi-Dzanga,7-Feb-20,s3://sentinel-s2-l2a/tiles/33/N/YC/2020/2/7/0,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,NaN
4,5,3.83151,17.75469,@300m,Central African Republic,License 171,7-Feb-20,s3://sentinel-s2-l2a/tiles/33/N/YC/2020/2/7/0,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,NaN
5,6,0.41152,15.47651,@300m,Republic of Congo,Outside of concession,8-Aug-20,s3://sentinel-s2-l2a/tiles/33/N/WA/2020/8/8/1,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,NaN
6,7,0.71847,21.00857,@500m,DRC,009/11,9-Jun-18,s3://sentinel-s2-l2a/tiles/34/N/DF/2018/6/9/0,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,NaN
7,8,1.88150,22.25298,@500m,DRC,52b/14,11-Feb-20,s3://sentinel-s2-l2a/tiles/34/N/GH/2020/2/11/0,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,NaN
8,9,1.72708,22.16947,@500m,DRC,52b/14,11-Feb-20,s3://sentinel-s2-l2a/tiles/34/N/GH/2020/2/11/0,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,Not visible on Google Earth Engine
9,10,2.65487,22.99404,@300m,DRC,Unnamed concession,11-Feb-20,s3://sentinel-s2-l2a/tiles/34/N/GH/2020/2/11/0,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,Too faint?


In [104]:
df["Comment"] = df["Comment"].where(pd.notnull(df["Comment"]), "blank")

In [110]:
df2 = df[~df["Comment"].str.contains("NB: mining")]

In [114]:
df3 = df2[17:]

In [115]:
unique_paths = df3.drop_duplicates(subset="tile AWS path")["tile AWS path"].tolist()[:-1]

In [119]:
len(df3)

24

In [116]:
len(unique_paths)

4

In [54]:
# Coulnd't download all Tiles

# bands = ['R10m/TCI']

# labelled_tiles = []
# for path in unique_paths:
#     Tilename = ''.join(path.split("/")[4:7])
#     Date = '-'.join(path.split("/")[7:10])
#     AmazonID = path.split("/")[-1]
#     labelled_tiles.append((Tilename,Date,AmazonID))

# df_results = pd.DataFrame(labelled_tiles, columns=["Tilename","Date","AmazonID"])
# output = list(df_results.itertuples(index=False,name=None))
# for tile in output:
#     tile_name, time, aws_index = tile
#     print(tile_name,time,aws_index)
#     data_folder = f'/Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha'
#     request = AwsTileRequest(
#         tile=tile_name,
#         time=time,
#         bands = bands, 
#         aws_index=aws_index,
#         data_folder=data_folder,
#         data_source=DataSource.SENTINEL2_L2A,
#         safe_format = True
#     )

#     request.save_data(redownload=True)

# Download Labelled Tiles

In [65]:
# file_paths = [s + "/R10m/TCI.jp2" for s in unique_paths]

In [117]:
unique_paths

['s3://sentinel-s2-l2a/tiles/34/N/DF/2020/4/14/0',
 's3://sentinel-s2-l2a/tiles/33/N/WA/2020/2/15/0',
 's3://sentinel-s2-l2a/tiles/33/N/UC/2020/1/24/0',
 's3://sentinel-s2-l2a/tiles/34/M/CD/2020/8/12/0']

In [118]:
import subprocess
for index,path in enumerate(unique_paths):
    subprocess.call(f"mkdir /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/{index}",shell=True)
    subprocess.call(f"aws s3 cp {path}/R10m/TCI.jp2 /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/{index} --request-payer",shell=True)
    subprocess.call(f"aws s3 cp {path}/metadata.xml /Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Labelled/Tiles_v1_Misha/{index} --request-payer",shell=True)
    